In [2]:
from sklearn.cluster import KMeans, k_means, Birch
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd

In [1]:
df = pd.read_csv('./datasets/backfill_mean_data.csv.csv')
df = df.drop(columns = 'Unnamed: 0')

NameError: name 'pd' is not defined

In [62]:
ss = StandardScaler()
df_scaled = ss.fit_transform(df)

In [63]:
model = KMeans(n_clusters= 4, random_state= 42)

In [64]:
model.fit(df_scaled)

KMeans(n_clusters=4, random_state=42)

In [65]:
labels = model.labels_

In [66]:
labels = model.labels_
centroids = model.cluster_centers_

Predicted clusters to points:  [3 3 3 ... 3 3 3]
Location of centroids: 
[[-1.79507378e-02 -1.61072901e-02  1.56859155e-02  3.60161896e-03
  -5.76335969e-02  2.14422507e-02 -5.91206769e-01 -1.49728720e-01
   1.76741413e-01 -3.65882241e-03 -5.91240091e-01 -1.48572166e-01
   1.74724975e-01 -3.65882241e-03 -1.05503137e-01  4.58251057e-02
  -1.05518914e-01  4.58251057e-02 -2.95751010e-01 -1.12652170e-01
  -8.22132909e-02 -7.41439327e-02  8.00493264e-02  0.00000000e+00
  -2.41202432e-01 -2.14422507e-02]
 [ 6.59347613e+01  6.59578641e+01 -6.59606100e+01  1.23777924e-02
   2.60549065e+00  2.14422507e-02  1.22977493e+00  9.27636286e-01
  -1.32873773e+00  8.75208571e-03  1.23487222e+00  9.29048573e-01
  -1.33176251e+00  8.75208571e-03  9.55014522e-01 -1.24933286e-01
   9.54825999e-01 -1.24933286e-01  7.95360353e-01 -1.82221108e-01
  -2.27939418e-01 -1.55273755e-01  1.52790007e-01  0.00000000e+00
   2.59899457e+00 -2.14422507e-02]
 [-1.70044473e-02 -1.61616930e-02  1.56423940e-02  1.23777924e-02

In [67]:
df['labels'] = labels
df.head()

,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,...,pl_bmassjlim,pl_orbeccen,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqtlim,labels
0,326.03000,0.32,-0.32,0.0,1.29,0.0,12.1,0.336,-0.336,0.0,...,0.0,0.231,0.0,99.27,6.24,-5.76,0.0,1700.0,0.0,3
1,516.21997,3.20,-3.20,0.0,1.53,0.0,12.3,0.336,-0.336,0.0,...,0.0,0.080,0.0,99.27,6.24,-5.76,0.0,1700.0,0.0,3
2,185.84000,0.23,-0.23,0.0,0.83,0.0,12.9,0.336,-0.336,0.0,...,0.0,0.000,0.0,99.27,6.24,-5.76,0.0,1700.0,0.0,3
3,1773.40002,2.50,-2.50,0.0,2.93,0.0,12.9,0.336,-0.336,0.0,...,0.0,0.370,0.0,99.27,6.24,-5.76,0.0,1700.0,0.0,3
4,798.50000,1.00,-1.00,0.0,1.66,0.0,13.5,0.336,-0.336,0.0,...,0.0,0.680,0.0,99.27,6.24,-5.76,0.0,1700.0,0.0,3


In [68]:
df.labels.unique()

array([3, 0, 2, 1], dtype=int32)

In [82]:
score = silhouette_score(df_scaled, labels, metric='euclidean')
score

0.4317274576064746

In [74]:
#trying a birch clustering model to see if our silhouette score can be improved
birch_model = Birch(n_clusters = 4)

In [75]:
birch_model.fit(df_scaled)

Birch(n_clusters=4)

In [77]:
df['birch_labels'] = birch_model.labels_

In [78]:
df['birch_labels'].unique()

array([0, 2, 1, 3])

In [85]:
birch_score = silhouette_score(df_scaled, birch_model.labels_, metric='euclidean')
birch_score
#our score is MUCH improved.  Love to see that.  We'll use these birch labels to build our classifier.

0.8985076623910687